1. В коде из методички реализуйте один или несколько из критериев останова (количество листьев, количество используемых признаков, глубина дерева и т.д.)

In [1]:
import numpy as np

In [2]:
# Построение дерева с помощью рекурсивной функции

def build_tree(data, labels, depth=0, max_depth=10):

    quality, t, index = find_best_split(data, labels)

    #  Базовый случай - прекращаем рекурсию, когда нет прироста в качества
    if quality == 0:
        return Leaf(data, labels)
    
    #  Останов по глубине дерева
    if depth >= max_depth:
        return Leaf(data, labels)

    true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
    depth += 1

    # Рекурсивно строим два поддерева
    true_branch = build_tree(true_data, true_labels, depth=depth)
    false_branch = build_tree(false_data, false_labels, depth=depth)

    # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
    return Node(index, t, true_branch, false_branch)

In [3]:
def find_best_split(data, labels, max_feature=10):
    
    #  обозначим минимальное количество объектов в узле
    min_leaf = 5

    current_gini = gini(labels)

    best_quality = 0
    best_t = None
    best_index = None
    
    n_features = data.shape[1]
    
    # отбор индексов неиспользуемых признаков
    if max_feature < n_features:
        pass_feature = np.random.choice(range(n_features), n_features - max_feature, replace=False)
    else:
        pass_feature = []
    
    for index in range(n_features):
        # исключение неиспользуемых признаков
        if index in pass_feature:
            continue
        
        # будем проверять только уникальные значения признака, исключая повторения
        t_values = np.unique([row[index] for row in data])
        
        for t in t_values:
            true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
            #  пропускаем разбиения, в которых в узле остается менее 5 объектов
            if len(true_data) < min_leaf or len(false_data) < min_leaf:
                continue
            
            current_quality = quality(true_labels, false_labels, current_gini)
            
            #  выбираем порог, на котором получается максимальный прирост качества
            if current_quality > best_quality:
                best_quality, best_t, best_index = current_quality, t, index

    return best_quality, best_t, best_index

2*. Реализуйте дерево для задачи регрессии. Возьмите за основу дерево, реализованное в методичке, заменив механизм предсказания в листе на взятие среднего значения по выборке, и критерий Джини на дисперсию значений.

In [4]:
# И класс терминального узла (листа)

class Leaf:
    
    def __init__(self, data, y):
        self.data = data
        self.y = y
        self.prediction = self.predict()
        
    def predict(self):
        # среднее значение в листе   
        prediction = np.mean(self.y)
        return prediction 

In [5]:
# Расчет дисперсии

def dispersion(y):
    dispersion = np.mean((y - np.mean(y))**2)
    return dispersion

In [6]:
# Расчет качества

def quality(left_y, right_y, current_dispersion):

    # доля выбоки, ушедшая в левое поддерево
    p = float(left_y.shape[0]) / (left_y.shape[0] + right_y.shape[0])
    
    return current_dispersion - p * dispersion(left_y) - (1 - p) * dispersion(right_y)